In [1]:
import numpy as np


def average_rank(rank: np.array) -> np.array:
    return -rank


def utility_fcn(gt_effect, rank, weight_fcn):
    return np.mean(gt_effect * weight_fcn(rank))

## Motivation

* Grouped Recommendation 의 경우 user를 그룹 단위로 분류하여, 같은 그룹에 속하는 유저에게는 아이템의 순위를 동일하게 결정.
* 두개의 Grouped Recommender 모형의 성능을 비교하는데 기존의 Average Utility 함수가 적합한 평가척도인가?

$$R(f) = \frac{1}{\mathcal{U}} \sum_{u=1}^{\mathcal{U}} U(\hat{r}_u, \tau_u) = \frac{1}{\mathcal{U} \cdot \mathcal{I}}\sum_{u=1}^{\mathcal{U}} \sum_{i=1}^{\mathcal{I}} \lambda(\hat{r}_{u,i}) \tau_{u,i}$$

우선, weighting function $\lambda$를 Average Rank 인 경우로 생각해보자.
$$\lambda^{AR}(\hat{r}_{u,i}) = -(\hat{r}_{u,i})$$

### (Case 1)

In [10]:
tau1 = np.array([1, -1, 1, 0, 0, 0,])
tau2 = np.array([0, 0, 0, 1, -1, 1,])
print(f"Average tau = {(tau1+tau2)/2}")

rank1 = np.array([1,6,2,3,4,5])
rank2 = np.array([4,5,3,2,6,1])

Average tau = [ 0.5 -0.5  0.5  0.5 -0.5  0.5]


* rank1은 첫번째 유저의 광고효과 tau1 을 기준으로 순위를 배열
* rank2는 광고효과의 평균을 기준으로 순위를 결정

In [11]:
eval_rank1 = np.mean([ 
    utility_fcn(tau1, rank1, average_rank), 
    utility_fcn(tau2, rank1, average_rank), 
    ])
print(f"Performance of Rank1: {eval_rank1}")

eval_rank2 = np.mean([ 
    utility_fcn(tau1, rank2, average_rank), 
    utility_fcn(tau2, rank2, average_rank), 
    ])
print(f"Performance of Rank2: {eval_rank2}")

Performance of Rank1: -0.08333333333333331
Performance of Rank2: 0.08333333333333334


* 그룹 내 유저의 utility 평균을 봤을 때, rank2이 성능이 더 좋음.
* 그러나, 첫번째 유저는 rank2로 주어진 추천결과를 만족할까? rank1를 더 만족하지 않을까?

### (Case 2)

In [17]:
tau3 = np.array([0, 1, -1, 1, 0, 0,])
tau4 = np.array([ 0, 0, 1, -1, 1,0,])
print(f"Average tau = {(tau3+tau4)/2}")

rank3 = np.array([3,2,6,1,5,4])
rank4 = np.array([3,1,5,6,2,4])
rank5 = np.array([5,2,3,4,1,6])


Average tau = [0.  0.5 0.  0.  0.5 0. ]


* Rank3은 첫번째 유저의 광고효과 기준
* Rank4, Rank5는 광고효과의 그룹내 평균을 기준
* 첫번째 유저의 입장에서는 Rank 3이 가장 적합한 순위이고, 음의 광고효과를 가지는 아이템의 위치를 기준으로 봤을 때 Rank4가 Rank5 보다 적합하다.
$$Rank3 > Rank4 > Rank5$$

In [18]:
eval_rank3 = np.mean([ 
    utility_fcn(tau3, rank3, average_rank), 
    utility_fcn(tau4, rank3, average_rank), 
    ])
print(f"Performance of Rank3: {eval_rank3}")

eval_rank4 = np.mean([ 
    utility_fcn(tau3, rank4, average_rank), 
    utility_fcn(tau4, rank4, average_rank), 
    ])
print(f"Performance of Rank4: {eval_rank4}")

eval_rank5 = np.mean([ 
    utility_fcn(tau3, rank5, average_rank), 
    utility_fcn(tau4, rank5, average_rank), 
    ])
print(f"Performance of Rank5: {eval_rank5}")

Performance of Rank3: -0.5833333333333334
Performance of Rank4: -0.25
Performance of Rank5: -0.25


* 그러나 평가지표로 봤을때는 
$$Rank5=Rank4>Rank3$$

* Grouped Recommender의 성능을 비교할 때 그나마 그룹 내 모두의 마음에 드는 Ranking이 잘 평가되도록 할 수 있을까?
* 기존 Grouped Recommender의 평가지표가 다른지 확인해보자.